In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ncx2
from scipy.signal import correlate
from matplotlib.lines import Line2D
from scipy.stats import lognorm
from scipy.stats import norm

In [6]:
# Parameters
dt = 1 
q = 0
rf = 0.03 
sigma = 0.2
beta = -2 #cev 1.5

# Compute the density of ST for different values of beta
#Build the ST grid
x0=1
xmin=0.01
xmax=4
npoints=400
xTgrid=np.linspace(xmin,xmax,npoints)#price grid
logxTgrid=np.log(xTgrid)#return grid
betas=np.linspace(-2,2,5) #beta values

In [7]:
class Constant_Elasticity_Of_Variance:
    """dS = mu*S*dt + sigma*S^(beta+1)*dW
    Model used to reproduce the volatility smile effect
    Requires numpy, pandas and plotly.express"""
    def __init__(self, mu, sigma, n_paths, n_steps, t, T, S_0, beta):
        self.mu = mu
        self.sigma = sigma
        self.n_paths = n_paths
        self.n_steps = n_steps
        self.t = t
        self.T =T
        self.S_0 = S_0
        self.beta = beta
    
    def get_paths(self):
        """Returns the paths, S, for the Constant Elasticity of Variance Process"""
        dt = self.T/self.n_steps
        dW = np.sqrt(dt)*np.random.randn(self.n_steps, self.n_paths)
        S = np.concatenate((self.S_0*np.ones((1, self.n_paths)), np.zeros((self.n_steps, self.n_paths))), axis=0) 
        
        for i in range(0, self.n_steps):
            S[i+1,:] = S[i,:] + self.mu*S[i,:]*dt + self.sigma*(S[i,:]**(self.beta+1))*dW[i,:]
            S[i+1,:] = np.maximum(S[i+1,:], np.zeros((1, self.n_paths)))
        
        return S
    
    def get_expectation(self):
        """Returns the expectation, E[S], for the Constant Elasticity of Variance Process"""
        ES = self.S_0*np.exp(self.mu*self.t)
        return ES
    
    def simulate(self, plot_expected=False):
        """Returns the plot of the random paths taken by the Constant Elasticity of Variance Process"""
        plotting_df = pd.DataFrame(self.get_paths())
        if plot_expected==True:
            plotting_df["Expected Path"]=self.get_expectation()
        fig = px.line(plotting_df, labels={"value":"Value of S", "variable":"Paths"})
        return fig.show()

In [ ]:
cev = Constant_Elasticity_Of_Variance()

In [5]:
# Calculate the expected path
EX = X0 * np.exp(-alpha * t) + mu * (1 - np.exp(-alpha * t))

# Begin plotting

plt.figure()
plt.plot(t, X[::1000, :].T, alpha=0.5)  # Plotting sample paths
plt.plot(t, EX, 'r-', label='Expected path',linewidth=2)
plt.plot(t, np.mean(X, axis=0), 'k--', label='Mean path',linewidth=2, markersize=6)
plt.plot(t, mu * np.ones_like(t), 'k:', label='Long-term average')

plt.legend()
plt.xlabel('t')
plt.ylabel('X')
plt.ylim([mu - 4 * (sigma / np.sqrt(2 * alpha)), mu + 4 * (sigma / np.sqrt(2 * alpha))])
plt.title(r'Ornstein-Uhlenbeck process $dX = \alpha(\mu-X)dt + \sigma dW$')
plt.show()

NameError: name 'X0' is not defined

In [ ]:
## Variance = Mean Square Deviation

# Calculate the theoretical variance of the Ornstein-Uhlenbeck process
# This follows the formula: sigma^2 / (2 * alpha) * (1 - exp(-2 * alpha * t))

VARX = sigma**2 / (2 * alpha) * (1 - np.exp(-2 * alpha * t))

# Plotting
plt.figure(figsize=(10, 6))

# Plot the theoretical variance as a red line
plt.plot(t, VARX, 'r', label='Theory')

# Plot the linear growth of variance over time (sigma^2 * t) as a green line
plt.plot(t, sigma**2 * t, 'g', label=r'$\sigma^2t$')

# Plot the asymptotic limit of variance (sigma^2 / (2 * alpha)) as a blue line
plt.plot(t, sigma**2 / (2 * alpha) * np.ones_like(t), 'b', label=r'$\sigma^2/(2\alpha)$')

# Plot the sampled variance (computed from the simulated paths) as a magenta line
# np.var calculates variance across rows (for each timestep), with ddof=1 for an unbiased estimator
plt.plot(t, np.var(X, axis=0, ddof=1), 'm', label='Sampled 1')

# Plot the mean squared deviation (Sampled 2) as a cyan dashed line
# This is calculated as the mean of the squared differences from the expected path
plt.plot(t, np.mean((X - EX)**2, axis=0), 'c--', label='Sampled 2')

plt.legend(loc='lower right')
plt.xlabel('t')
plt.ylabel(r'Var(X) = E$((X-E(X))^2)$')
plt.ylim([0, 0.0006])
plt.title(r'Ornstein-Uhlenbeck process: variance')
plt.show()


In [ ]:
##4A Mean Absolute Deviation


# Plotting
plt.figure(figsize=(10, 6))

# Calculate and plot the theoretical Mean Absolute Deviation (MAD) as a red line
# The formula for theoretical MAD in the Ornstein-Uhlenbeck process is:
# sigma * sqrt((1 - exp(-2 * alpha * t)) / (pi * alpha))

plt.plot(t, sigma * np.sqrt((1 - np.exp(-2 * alpha * t)) / (np.pi * alpha)), 'r', label='Theory')

# Plot the linear growth of MAD over time (sigma * sqrt(2 * t / pi)) as a green line
plt.plot(t, sigma * np.sqrt(2 * t / np.pi), 'g', label=r'$\sigma(2t/\pi)^{1/2}$')

# Plot the asymptotic limit of MAD (sigma / sqrt(pi * alpha)) as a blue line
plt.plot(t, sigma / np.sqrt(np.pi * alpha) * np.ones_like(t), 'b', label='Long-term average')

# Plot the sampled Mean Absolute Deviation (MAD) as a magenta line
# This is calculated as the mean of the absolute deviations from the expected path
plt.plot(t, np.mean(np.abs(X - EX), axis=0), 'm', label='Sampled')

# Add legend, labels, and set y-axis limits
plt.legend(loc='lower right')
plt.xlabel('t')
plt.ylabel(r'E$|X-E(X)| = (2Var(X)/\pi)^{1/2}$')
plt.ylim([0, 0.02])

# Title of the plot, using LaTeX for mathematical symbols
plt.title('Ornstein-Uhlenbeck process: mean absolute deviation')

# Display the plot
plt.show()


In [ ]:
#### 5A Autocovariance
from scipy.signal import correlate

# Initialize the autocovariance array
C = np.zeros((npaths, 2 * nsteps + 1))

# Calculate the autocovariance for each path
for i in range(npaths):
    deviation = X[i, :] - EX
    # Compute autocorrelation and normalize by the number of steps
    C[i, :] = correlate(deviation, deviation, mode='full') / nsteps



# Average over all paths
C = np.mean(C, axis=0)

# Begin plotting
plt.figure(figsize=(10, 6))

# Plot theoretical autocovariance
# The formula is: sigma^2 / (2 * alpha) * exp(-alpha * t)
plt.plot(t, sigma**2 / (2 * alpha) * np.exp(-alpha * t), 'r', label='Theory')

# Plot sampled autocovariance
# Only plotting the second half since it's symmetric and the first half corresponds to negative lags
plt.plot(t, C[nsteps:], 'b', label='Sampled')


# Plot variance for infinite t (sigma^2 / (2 * alpha))
plt.plot(0, sigma**2 / (2 * alpha), 'ro', label ='Theory Variance for Infinite t')  # Var for infinite t

# Plot average sampled variance
plt.plot(0, np.mean(np.var(X, axis=0, ddof=0)), 'bo', label='Average Sampled Var', linewidth=1.5)


# Add labels, legend, and title
plt.xlabel(r'$\tau$')
plt.ylabel(r'C($\tau$)')
plt.legend(loc='upper right')
plt.title('Ornstein-Uhlenbeck process: autocovariance')

# Display the plot
plt.show()

In [ ]:
## 6A Autocorrelation

# The autocorrelation is the Covariance/Variance. However, since our OUP is
# only quasi-stationary (i.e. it is only stationary in the limit t -> inf)
# we will compute the autocorrelation as we have done above, in the limit
# as t -> inf

# It can be shown that in the limit, the autocorrelation becomes
# Corr(t,s) = exp(-1*alpha*tau)     with t < s

# Theoretical autocorrelation
CORRX = np.exp(-alpha * t)

# Begin plotting
plt.figure(figsize=(10, 6))

# Plot theoretical autocorrelation as a red line
plt.plot(t, CORRX, 'r', label='Theory')

# Plot sampled autocorrelation
# The sampled autocorrelation is computed as the autocovariance divided by the variance at time 0 (C[nsteps+1])
# Here, we use the second half of C (corresponding to non-negative lags)
plt.plot(t, C[nsteps:] / C[nsteps], 'b', label='Sampled')

# Add labels, legend, and title
plt.xlabel(r'$\tau$')
plt.ylabel(r'c($\tau$)')
plt.legend()
plt.title('Ornstein-Uhlenbeck process: autocorrelation')

# Display the plot
plt.show()
